# Data Analysis Sample Code

This was a class assignment from the data science programming course at UC Berkeley. I received a modified version of a VCF file, which included a tab-delimited file with some values including key-value pair data, and was asked to parse it and find meaningful information. 

Skills I am trying to demonstrate are:
* Data cleaning
* Data exploration and validation
* Analysis and reasoning 
* Applying functions on data frames
* Working with key-value pairs, even if they look a little bit unconventional 
* Dealing with missing data 

## Premise: Clinical disease data 

> Your boss comes to you Monday morning and says “I figured out our next step; we are going to pivot from an online craft store and become a data center for genetic disease information! I found **ClinVar** which is a repository that contains expert curated data, and it is free for the taking. This is a gold mine! Look at the file and tell me what gene and mutation combinations are classified as dangerous.”  
>  
> Make sure that you only give your boss the dangerous mutations and include:  
>  
>    1) Gene name  
>    2) Mutation ID number  
>    3) Mutation Position (chromosome & position)  
>    4) Mutation value (reference & alternate bases)  
>    5) Clinical severity   
>    6) Disease that is implicated  
>  
> Use the instructor-modified "clinvar_final.txt". This file was modified to be not exactly the same as 'standard' .vcf file to test your data parsing skills. **This is a large file so do NOT upload it into your github repo!** Replace missing values in the dataframe with: 'Not_Given' and state in your answer how you define harmful mutations

### VCF file description (Summarized from version 4.1)

```
* The VCF specification:

VCF is a text file format which contains meta-information lines, a header line, and then data lines each containing information about a position in the genome. The format also can contain genotype information on samples for each position.

* Fixed fields:

There are 8 fixed fields per record. All data lines are **tab-delimited**. In all cases, missing values are specified with a dot (‘.’). 

1. CHROM - chromosome number
2. POS - position DNA nuceleotide count (bases) along the chromosome
3. ID - The unique identifier for each mutation
4. REF - reference base(s)
5. ALT - alternate base(s)
6. FILTER - filter status
7. QUAL - quality
8. INFO - a semicolon-separated series of keys with values in the format: <key>=<data>

```
### Applicable INFO field specifications

```
GENEINFO = <Gene name>
CLNSIG =  <Clinical significance>
CLNDN = <Disease name>
```

### Sample ClinVar data (vcf file format - not exactly the same as the file to download!)

```
##fileformat=VCFv4.1
##fileDate=2019-03-19
##source=ClinVar
##reference=GRCh38							
#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO
1	949523	rs786201005	C	T	.	.	GENEINFO=ISG15;CLNSIG=5
1	949696	rs672601345	C	CG	.	.	GENEINFO=ISG15;CLNSIG=5;CLNDBN=Cancer
1	949739	rs672601312	G	T	.	.	GENEINFO=ISG15;CLNDBN=Cancer
1	955597	rs115173026	G	T	.	.	GENEINFO=AGRN;CLNSIG=2; CLNDBN=Cancer
1	955619	rs201073369	G	C	.	.	GENEINFO=AGG;CLNDBN=Heart_dis 
1	957640	rs6657048	C	T	.	.	GENEINFO=AGG;CLNSIG=3;CLNDBN=Heart_dis 
1	976059	rs544749044	C	T	.	.	GENEINFO=AGG;CLNSIG=0;CLNDBN=Heart_dis 
```

In [12]:
# 4) Your code here

# See the cells below for my data exploration. 

# Response

### Data Import

In [13]:
# Let's look at the first few lines of my data. Change the number until I can get a decent picture. 

!head -30 clinvar_final.txt

#fileformat=VCFv4.1
#fileDate=2019-03-01
#source=ClinVar
#reference=GRCh38
#ID=<Description="ClinVar Variation ID">
#INFO=<ID=AF_ESP,Number=1,Type=Float,Description="allele frequencies from GO-ESP">
#INFO=<ID=AF_EXAC,Number=1,Type=Float,Description="allele frequencies from ExAC">
#INFO=<ID=AF_TGP,Number=1,Type=Float,Description="allele frequencies from TGP">
#INFO=<ID=ALLELEID,Number=1,Type=Integer,Description="the ClinVar Allele ID">
#INFO=<ID=CLNDN,Number=.,Type=String,Description="ClinVar's preferred disease name for the concept specified by disease identifiers in CLNDISDB">
#INFO=<ID=CLNDNINCL,Number=.,Type=String,Description="For included Variant : ClinVar's preferred disease name for the concept specified by disease identifiers in CLNDISDB">
#INFO=<ID=CLNDISDB,Number=.,Type=String,Description="Tag-value pairs of disease database name and identifier, e.g. OMIM:NNNNNN">
##INFO=<ID=CLNDISDBINCL,Number=.,Type=String,Description="For included Variant: Tag-value pairs of disease databa

In [14]:
# Last few rows 
!tail -5 clinvar_final.txt

3	179210507	403908	A	G	.	.	ALLELEID=393412;CLNDISDB=MedGen:C0018553,Orphanet:ORPHA201,SNOMED_CT:58037000;CLNDN=Cowden_syndrome;CLNHGVS=NC_000003.12:g.179210507A>G;CLNREVSTAT=criteria_provided,_single_submitter;CLNSIG=Uncertain_significance;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GENEINFO=PIK3CA:5290;MC=SO:0001583|missense_variant;ORIGIN=1;RS=1060500027
3	179210511	526648	T	C	.	.	ALLELEID=519163;CLNDISDB=MedGen:C0018553,Orphanet:ORPHA201,SNOMED_CT:58037000;CLNDN=Cowden_syndrome;CLNHGVS=NC_000003.12:g.179210511T>C;CLNREVSTAT=criteria_provided,_single_submitter;CLNSIG=Likely_benign;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GENEINFO=PIK3CA:5290;MC=SO:0001819|synonymous_variant;ORIGIN=1;RS=1480813252
3	179210515	526640	A	C	.	.	AF_EXAC=0.00002;ALLELEID=519178;CLNDISDB=MedGen:C0018553,Orphanet:ORPHA201,SNOMED_CT:58037000|MedGen:C0027672,SNOMED_CT:699346009;CLNDN=Cowden_syndrome|Hereditary_cancer-predisposing_syndrome;CLNHGVS=NC_000003.12:g.179210515A>C;CLNREVSTAT=criteria_

In [15]:
# Check field names
!head -28 clinvar_final.txt | tail -1

CHROM	POS	ID	REF	ALT	FILTER	QUAL	INFO


#### Observations about the input file

* The first 27 rows are informative, but aren't actually the data that we want to import. I noticed that these all start with a single hashtag/pound sign, "#". When we import, we would want to exclude all of these lines. 

* Line 28 is the header. We want to grab these and assign to our pandas dataframe later.

* Line 29 onward appears to be the data that we want. 

* The last lines of the dataset seem like they're still part of the data we want to analyze. 

* The last column, called INFO, has an array inside this single column (when delmiiting each line by tab) and this array contains multiple keys and values. The values in here are parsed by a semicolon, ';'. We should parse this to get more info later. The data dictionary for this exists in the first few rows of the file. 

In [16]:
# Import file, then close the file. 
# Only fetch the lines that do NOT start with a pound sign, "#"
with open('Genomics_Question.txt', 'r') as f:
    lines = [l for l in f if not l.startswith('#')]
    

In [17]:
# 1 - see raw file
print("See how when I just open the raw file, each line is a separate item of a list.\
\nAlso I can see \\t for each tab, etc.\n")
print(lines)
print("\n\n")

print("By joining each line in the list, I get a nice, single string that is more readable.\
\nThis is the data that we want to analyze.\n")
# 2 - Join lines to produce a single string of all the data. 
print(''.join(lines))

See how when I just open the raw file, each line is a separate item of a list.
Also I can see \t for each tab, etc.

['1\t949523\trs786201005\tC\tT\t.\t.\tGENEINFO=ISG15;CLNSIG=5\n', '1\t949696\trs672601345\tC\tCG\t.\t.\tGENEINFO=ISG15;CLNSIG=5;CLNDBN=Cancer\n', '1\t949739\trs672601312\tG\tT\t.\t.\tGENEINFO=ISG15;CLNDBN=Cancer\n', '1\t955597\trs115173026\tG\tT\t.\t.\tGENEINFO=AGRN;CLNSIG=2; CLNDBN=Cancer\n', '1\t955619\trs201073369\tG\tC\t.\t.\tGENEINFO=AGG;CLNDBN=Heart_dis \n', '1\t957640\trs6657048\tC\tT\t.\t.\tGENEINFO=AGG;CLNSIG=3;CLNDBN=Heart_dis \n', '1\t976059\trs544749044\tC\tT\t.\t.\tGENEINFO=AGG;CLNSIG=0;CLNDBN=Heart_dis ']



By joining each line in the list, I get a nice, single string that is more readable.
This is the data that we want to analyze.

1	949523	rs786201005	C	T	.	.	GENEINFO=ISG15;CLNSIG=5
1	949696	rs672601345	C	CG	.	.	GENEINFO=ISG15;CLNSIG=5;CLNDBN=Cancer
1	949739	rs672601312	G	T	.	.	GENEINFO=ISG15;CLNDBN=Cancer
1	955597	rs115173026	G	T	.	.	GENEINFO=AGRN;CLNSI

In [18]:
import io

# 3 - read_csv 
pd.read_csv(
    # from step 2. I was wondering what this did... 
    io.StringIO(''.join(lines)),  
        # turns out, if I just pass in the data in this argument, read_csv will try to find a file with that name.
        # So to tell it that we're not chasing down a file of this name, 
        # but rather, this string IS the file we want to parse, 
        # we use io.StringIO. 
    
    # Assign column names. The sample data doesn't have headers. 
    names=["CHROM", "POS", "ID", "REF", "ALT", "FILTER", "QUAL", "INFO"], 
    
    # Assign data types 
    dtype={'CHROM': str, 'POS': str, 'ID': str, 'REF': str,  
           'ALT': str, 'FILTER': str, 'QUAL': str, 'INFO': str}, # watch the column order 
    sep='\t'
)



,CHROM,POS,ID,REF,ALT,FILTER,QUAL,INFO
0,1,949523,rs786201005,C,T,.,.,GENEINFO=ISG15;CLNSIG=5
1,1,949696,rs672601345,C,CG,.,.,GENEINFO=ISG15;CLNSIG=5;CLNDBN=Cancer
2,1,949739,rs672601312,G,T,.,.,GENEINFO=ISG15;CLNDBN=Cancer
3,1,955597,rs115173026,G,T,.,.,GENEINFO=AGRN;CLNSIG=2; CLNDBN=Cancer
4,1,955619,rs201073369,G,C,.,.,GENEINFO=AGG;CLNDBN=Heart_dis
5,1,957640,rs6657048,C,T,.,.,GENEINFO=AGG;CLNSIG=3;CLNDBN=Heart_dis
6,1,976059,rs544749044,C,T,.,.,GENEINFO=AGG;CLNSIG=0;CLNDBN=Heart_dis


In [19]:
# Okay, so now let's import the real data.

with open('clinvar_final.txt', 'r') as f:
    lines = [l for l in f if not l.startswith('#')]

clin = pd.read_csv(
    # from step 2. I was wondering what this did... 
    io.StringIO(''.join(lines)),  
        # turns out, if I just pass in the data in this argument, read_csv will try to find a file with that name.
        # So to tell it that we're not chasing down a file of this name, 
        # but rather, this string IS the file we want to parse, 
        # we use io.StringIO. 
    
    # No need to assign column names to the real data - the header's already in there
    
    # Assign data types 
    dtype={'CHROM': str, 'POS': str, 'ID': str, 'REF': str,  
           'ALT': str, 'FILTER': str, 'QUAL': str, 'INFO': str}, # watch the column order 
    sep='\t'
)



In [20]:
clin.head(5)

,CHROM,POS,ID,REF,ALT,FILTER,QUAL,INFO
0,1,1014O42,475283,G,A,.,.,AF_ESP=0.00546;AF_EXAC=0.00165;AF_TGP=0.00619;...
1,1,1O14122,542074,C,T,.,.,AF_ESP=0.00015;AF_EXAC=0.00010;ALLELEID=514926...
2,1,1014143,183381,C,T,.,.,"ALLELEID=181485;CLNDISDB=MedGen:C4015293,OMIM:..."
3,1,1014179,542075,C,T,.,.,"ALLELEID=514896;CLNDISDB=MedGen:C4015293,OMIM:..."
4,1,1014217,475278,C,T,.,.,AF_ESP=0.00515;AF_EXAC=0.00831;AF_TGP=0.00339;...


In [21]:
# Let's save a copy of clin just in case I mess it up.

import copy
clin_backup = copy.deepcopy(clin) 
# can't remember if a df is mutable, so I'm just gonna take a safer route by doing a deep copy

In [22]:
clin_backup.head()

,CHROM,POS,ID,REF,ALT,FILTER,QUAL,INFO
0,1,1014O42,475283,G,A,.,.,AF_ESP=0.00546;AF_EXAC=0.00165;AF_TGP=0.00619;...
1,1,1O14122,542074,C,T,.,.,AF_ESP=0.00015;AF_EXAC=0.00010;ALLELEID=514926...
2,1,1014143,183381,C,T,.,.,"ALLELEID=181485;CLNDISDB=MedGen:C4015293,OMIM:..."
3,1,1014179,542075,C,T,.,.,"ALLELEID=514896;CLNDISDB=MedGen:C4015293,OMIM:..."
4,1,1014217,475278,C,T,.,.,AF_ESP=0.00515;AF_EXAC=0.00831;AF_TGP=0.00339;...


###  Data Understanding & Cleaning

What does our data look like?



In [23]:
clin.shape

(102321, 8)

In [24]:
clin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102321 entries, 0 to 102320
Data columns (total 8 columns):
CHROM     102321 non-null object
POS       102321 non-null object
ID        102321 non-null object
REF       102321 non-null object
ALT       102321 non-null object
FILTER    102321 non-null object
QUAL      102321 non-null object
INFO      102321 non-null object
dtypes: object(8)
memory usage: 6.2+ MB


Okay, so we've got 102 thousand rows. Got 8 columns as expected. 

Let's dig deeper. 

Copy paste the data dictionary here for easy reference. This will help me identify the right field types. 

1. CHROM - chromosome number
2. POS - position DNA nuceleotide count (bases) along the chromosome
3. ID - The unique identifier for each mutation
4. REF - reference base(s)
5. ALT - alternate base(s)
6. FILTER - filter status
7. QUAL - quality
8. INFO - a semicolon-separated series of keys with values in the format: `<key>=<data>`

#### Based on this information and the data dictionary from the VCF pdf file, here are my expectations.
- CHROM should be integer
- POS should be integer
- ID should be unique - if each row is mutation 
- For REF, reference base: Each base must be one of A,C,G,T,N (case insensitive). Multiple bases are permitted.
    - The value in the POS field refers to the position of the first base in the String.
-  ALT: Comma separated list of alternate non-reference alleles. Options are base Strings made up of the bases A,C,G,T,N, (case insensitive) or an angle-bracketed ID String (```“<ID>”```) or a breakend replacement string
- FILTER - filter status: PASS if this position has passed all filters, i.e. a call is made at this position. Otherwise, if the site has not passed all filters, a semicolon-separated list of codes for filters that fail.
- QUAL - quality: Phred-scaled quality score
    
- INFO - must parse later 
- A single period means NULL actually.

In [25]:
# Is every value in CHROM a number?
print(clin["CHROM"].str.isnumeric().unique())
# And an integer at that?
print(clin["CHROM"].str.isdigit().unique())


[ True]
[ True]


In [26]:
# Great. I can change it to integer if I want to. 

# clin.CHROM.astype(int) # this works without throwing an error
clin.CHROM = clin.CHROM.astype(int)

In [27]:
clin.CHROM.describe()

count    102321.000000
mean          1.880113
std           0.706169
min           1.000000
25%           1.000000
50%           2.000000
75%           2.000000
max           3.000000
Name: CHROM, dtype: float64

Okay, so the values here are always 1, 2, or 3. 

In [28]:
# Just confirm against the backup file
clin_backup.CHROM.unique()

array(['1', '2', '3'], dtype=object)

#### POS - is everything a number?

In [29]:
# Is every value in POS a number?
clin["POS"].str.isnumeric().unique()

array([False,  True])

In [30]:
# Nope. There are some POS values that are not numeric. Let's see them.
clin[~clin["POS"].str.isnumeric()]

,CHROM,POS,ID,REF,ALT,FILTER,QUAL,INFO
0,1,1014O42,475283,G,A,.,.,AF_ESP=0.00546;AF_EXAC=0.00165;AF_TGP=0.00619;...
1,1,1O14122,542074,C,T,.,.,AF_ESP=0.00015;AF_EXAC=0.00010;ALLELEID=514926...


In [31]:
# There is a random O in the number. Maybe it was an OCR problem. 
# I'm gonna assume that they're supposed to be 0, so replace them with 0.

clin[~clin["POS"].str.isnumeric()]["POS"].replace("o", "0")


0    1014O42
1    1O14122
Name: POS, dtype: object

In [32]:
# Nothing happened.
# Okay so they're not a letter O. I'll come back and deal with this later.

weird_pos = clin["POS"][0:5]

print(weird_pos)

for val in weird_pos:
    print(val.encode('utf-16'))

0    1014O42
1    1O14122
2    1014143
3    1014179
4    1014217
Name: POS, dtype: object
b'\xff\xfe1\x000\x001\x004\x00O\x004\x002\x00'
b'\xff\xfe1\x00O\x001\x004\x001\x002\x002\x00'
b'\xff\xfe1\x000\x001\x004\x001\x004\x003\x00'
b'\xff\xfe1\x000\x001\x004\x001\x007\x009\x00'
b'\xff\xfe1\x000\x001\x004\x002\x001\x007\x00'


In [33]:
# I don't nkow how to replace it, but I can at least have it be described. 
# See if it sheds any light. 

clin[clin["POS"].str.isnumeric()]["POS"].astype(int).describe()

count    1.023190e+05
mean     1.176866e+08
std      7.567756e+07
min      2.722230e+05
25%      4.670155e+07
50%      1.222615e+08
75%      1.787078e+08
max      2.484886e+08
Name: POS, dtype: float64

Yeah they're numbers. Didn't get much insight out of this other than that the number of digits range from 5 to 8. 

#### Is ID unique per row?

In [34]:
# Is ID unique?

assert len(clin["ID"]) - len(clin["ID"].unique()) == 0, \
"The difference of the total number of values and the number of unique ID values should be 0."


Yup - ID values are unique.

#### REF and ALT

In [35]:
# Let's see REF and ALT.  
clin[["REF", "ALT"]].drop_duplicates()[:20]

,REF,ALT
0,G,A
1,C,T
8,C,CG
9,G,T
12,G,C
13,C,A
15,G,GGCCGCTGCT
16,C,G
17,CG,GT
23,A,G


There are lots of unique values here. I can parse this out to see if the values here really contain just ACGATN. 

In [36]:
# Does REF and ALT only contain ACGTAN? Otherwise, are there any unexpeced values?

def unexpected_base(base):
    for letter in base:
        if letter not in "ACGATN":
            return True # As in TRUE, there ARE unexpected values
            continue
        
    return False # As in, no, there are NO unexpected values. 
    
# Test this out
print(unexpected_base("AAAAG"))
print(unexpected_base("."))
print(unexpected_base("ABACADG"))

False
True
True


In [37]:
# Let's try it across the whole column. 

clin["REF"].apply(unexpected_base).head()

0    False
1    False
2    False
3    False
4    False
Name: REF, dtype: bool

In [38]:
clin[clin["REF"].apply(unexpected_base)].head()

,CHROM,POS,ID,REF,ALT,FILTER,QUAL,INFO


Looks like every value in "REF" is one of ACGTN.

Check the ALT as well.

In [39]:
# Does ALT have any characters other than ACGTN or a period "." for NULL?
clin[(clin["ALT"].apply(unexpected_base)) & ~(clin["ALT"]==".")].head()

,CHROM,POS,ID,REF,ALT,FILTER,QUAL,INFO


No. So the values in ALT and REF look as expected. 

#### FILTER values.

Learned from the data dictionary that there's some sort of filter being applied to the data. If the datapoint passed all of them, the value in this field will be a PASS. So let's take a closer look.

In [40]:
print(clin.FILTER.unique())

['.']


It's all NULL. I'm so glad I can deal with one fewer problem.

Next.

#### QUAL


In [41]:
print(clin.QUAL.unique())

['.']


Also all NULL. So I'll ignore this. 

#### Parsing and exploring INFO. 

INFO contains data that can be parsed and could probably tell us about the dangers.
Let's take a look.

In [42]:
clin.INFO.unique()[:5]

array(['AF_ESP=0.00546;AF_EXAC=0.00165;AF_TGP=0.00619;ALLELEID=446939;CLNDISDB=MedGen:C4015293,OMIM:616126,Orphanet:ORPHA319563;CLNDN=Immunodeficiency_38_with_basal_ganglia_calcification;CLNHGVS=NC_000001.11:g.1014042G>A;CLNREVSTAT=criteria_provided,_single_submitter;CLNSIG=Benign;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GENEINFO=ISG15:9636;MC=SO:0001583|missense_variant;ORIGIN=1;RS=143888043',
       'AF_ESP=0.00015;AF_EXAC=0.00010;ALLELEID=514926;CLNDISDB=MedGen:C4015293,OMIM:616126,Orphanet:ORPHA319563;CLNDN=Immunodeficiency_38_with_basal_ganglia_calcification;CLNHGVS=NC_000001.11:g.1014122C>T;CLNREVSTAT=criteria_provided,_single_submitter;CLNSIG=Uncertain_significance;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GENEINFO=ISG15:9636;MC=SO:0001583|missense_variant;ORIGIN=1;RS=150861311',
       'ALLELEID=181485;CLNDISDB=MedGen:C4015293,OMIM:616126,Orphanet:ORPHA319563;CLNDN=Immunodeficiency_38_with_basal_ganglia_calcification;CLNHGVS=NC_000001.11:g.1014143C>T;CLNREVST

That's unreadable. Let's parse it.

In [43]:
# First stab at parsing this info. 
all_info_field = []

for x in clin.INFO:
    each_field = x.split(';')
    
    all_info_field.extend(each_field)
    
    
all_info_field[:10]

['AF_ESP=0.00546',
 'AF_EXAC=0.00165',
 'AF_TGP=0.00619',
 'ALLELEID=446939',
 'CLNDISDB=MedGen:C4015293,OMIM:616126,Orphanet:ORPHA319563',
 'CLNDN=Immunodeficiency_38_with_basal_ganglia_calcification',
 'CLNHGVS=NC_000001.11:g.1014042G>A',
 'CLNREVSTAT=criteria_provided,_single_submitter',
 'CLNSIG=Benign',
 'CLNVC=single_nucleotide_variant']

Each parsed value inside INFO is similar do a dictionary. According to the data dictionary, the values here are encoded as a semicolon-separated series of short keys with optional values in the format: `<key>=<data>[,data].`

There are so many types in here, though. Which ones do I care about? The instructions for the exam said, CLNSIG, CLNDN, and GENEINFO. 

In [44]:
# Anything else to care about? Let's import the additional information. Let's just look at it. What do you see?
with open('clinvar_final.txt', 'r') as f:
    add_line = [l for l in f if l.startswith('#')]

# Is there any way to convert this into a dictionary relaly quickly?
for line in add_line:
    if "<" in line:
        meat = line.split("<")[1].replace(">", "").replace("\n", "").replace("#", "")
        idname = meat.split(",")[0].split("=")[1]
        desc = meat.split("Description=")[1]
        
        print("|", idname, "|", desc, "|")
    


| "ClinVar Variation ID" | "ClinVar Variation ID" |
| AF_ESP | "allele frequencies from GO-ESP" |
| AF_EXAC | "allele frequencies from ExAC" |
| AF_TGP | "allele frequencies from TGP" |
| ALLELEID | "the ClinVar Allele ID" |
| CLNDN | "ClinVar's preferred disease name for the concept specified by disease identifiers in CLNDISDB" |
| CLNDNINCL | "For included Variant : ClinVar's preferred disease name for the concept specified by disease identifiers in CLNDISDB" |
| CLNDISDB | "Tag-value pairs of disease database name and identifier, e.g. OMIM:NNNNNN" |
| CLNDISDBINCL | "For included Variant: Tag-value pairs of disease database name and identifier, e.g. OMIM:NNNNNN" |
| CLNHGVS | "Top-level (primary assembly, alt, or patch) HGVS expression." |
| CLNREVSTAT | "ClinVar review status for the Variation ID" |
| CLNSIG | "Clinical significance for this single variant" |
| CLNSIGCONF | "Conflicting clinical significance for this single variant" |
| CLNSIGINCL | "Clinical significance for a hap

Okay, that's the best i can do with the time I have. Copy paste into an easier format.  


|ID|Description|
|---|---|
| AF_ESP | "allele frequencies from GO-ESP" |
| AF_EXAC | "allele frequencies from ExAC" |
| AF_TGP | "allele frequencies from TGP" |
| ALLELEID | "the ClinVar Allele ID" |
| CLNDN | "ClinVar's preferred disease name for the concept specified by disease identifiers in CLNDISDB" |
| CLNDNINCL | "For included Variant : ClinVar's preferred disease name for the concept specified by disease identifiers in CLNDISDB" |
| CLNDISDB | "Tag-value pairs of disease database name and identifier, e.g. OMIM:NNNNNN" |
| CLNDISDBINCL | "For included Variant: Tag-value pairs of disease database name and identifier, e.g. OMIM:NNNNNN" |
| CLNHGVS | "Top-level (primary assembly, alt, or patch) HGVS expression." |
| CLNREVSTAT | "ClinVar review status for the Variation ID" |
| CLNSIG | "Clinical significance for this single variant" |
| CLNSIGCONF | "Conflicting clinical significance for this single variant" |
| CLNSIGINCL | "Clinical significance for a haplotype or genotype that includes this variant. Reported as pairs of VariationID:clinical significance." |
| CLNVC | "Variant type" |
| CLNVCSO | "Sequence Ontology id for variant type" |
| CLNVI | "the variant's clinical sources reported as tag-value pairs of database and variant identifier" |
| DBVARID | "nsv accessions from dbVar for the variant" |
| GENEINFO | "Gene(s) for the variant reported as gene symbol:gene id. The gene symbol and id are delimited by a colon (:) and each pair is delimited by a vertical bar (|)" |
| MC | "comma separated list of molecular consequence in the form of Sequence Ontology ID|molecular_consequence" |
| ORIGIN | "Allele origin. One or more of the following values may be added: 0 - unknown; 1 - germline; 2 - somatic; 4 - inherited; 8 - paternal; 16 - maternal; 32 - de-novo; 64 - biparental; 128 - uniparental; 256 - not-tested; 512 - tested-inconclusive; 1073741824 - other" |
| RS | "dbSNP ID (i.e. rs number)" |
| SSR | "Variant Suspect Reason Codes. One or more of the following values may be added: 0 - unspecified, 1 - Paralog, 2 - byEST, 4 - oldAlign, 8 - Para_EST, 16 - 1kg_failed, 1024 - other" |



I don't know what a lot of these mean, so I'll have to take a guess.

- everything starting with AF_ seems to indicate where the data came from. 
- What's dangerous? Diseases are dangerous. CLNDISDB seems to be the disease identifier. Based on this, CLNDN and CLNDNINCL will show the ClinVar's preferred disease name. 
- CLNREVSTAT is the review status. I guess if we didn't review a mutation, then we can't know if it's dangerous or not.
- Not sure what clinical significance is.
- CLNVC, variant type, may have something interesting. What type is that? 
- CLNVCSO is the Sequence Ontology id. Ontology means study of cancer. So does having this ID mean that it's related to cancer?

And out of these, we really care about:
* GENEINFO = Gene name
* CLNSIG =  Clinical significance
* CLNDN = Disease name

Okay, so let's try to parse the data and see what's in these fields.

#### Extracting values from INFO
* GENEINFO = Gene name
* CLNSIG =  Clinical significance
* CLNDN = Disease name

In [45]:
# Create a function that'll process each cell 
# and create a dictionary where dict[ID] = value

def extract_info(cell):
    result = {}
    for x in cell.split(";"):
        result[x.split("=")[0]] = x.split("=")[1]
    return result

# Top 10 peek
clin.INFO[0:10].map(extract_info)



0    {'AF_ESP': '0.00546', 'AF_EXAC': '0.00165', 'A...
1    {'AF_ESP': '0.00015', 'AF_EXAC': '0.00010', 'A...
2    {'ALLELEID': '181485', 'CLNDISDB': 'MedGen:C40...
3    {'ALLELEID': '514896', 'CLNDISDB': 'MedGen:C40...
4    {'AF_ESP': '0.00515', 'AF_EXAC': '0.00831', 'A...
5    {'AF_ESP': '0.40158', 'AF_EXAC': '0.37025', 'A...
6    {'ALLELEID': '556509', 'CLNDISDB': 'MedGen:C40...
7    {'ALLELEID': '556512', 'CLNDISDB': 'MedGen:C40...
8    {'ALLELEID': '171289', 'CLNDISDB': 'MedGen:C40...
9    {'AF_EXAC': '0.00001', 'ALLELEID': '171288', '...
Name: INFO, dtype: object

In [46]:
# If I want to extract just the value of CLNSIG, this is how to do it on a single cell:

extract_info(clin.INFO[0])["CLNSIG"]


'Benign'

In [47]:
# Okay, let's put these dictionaries into a column. 

extracted_info = pd.DataFrame({"INFO_EXT": clin.INFO.map(extract_info)})


type(extracted_info)

pandas.core.frame.DataFrame

In [ ]:
clin_v2 = pd.concat([clin, extracted_info], axis=1)
clin_v2

In [49]:
# Cool. Now that the INFO field has been parsed... create extra columns that the boss wants
    # GENEINFO
    # CLNSIG
    # CLNDN

# To extract a single value in the dictionary in INFO_EXT
# clin_v2.INFO_EXT[0]["CLNDN"]

clin_v2["CLNDN"] = clin_v2.INFO_EXT.map(lambda x: x.get('CLNDN'))

clin_v2.head()

,CHROM,POS,ID,REF,ALT,FILTER,QUAL,INFO,INFO_EXT,CLNDN
0,1,1014O42,475283,G,A,.,.,AF_ESP=0.00546;AF_EXAC=0.00165;AF_TGP=0.00619;...,"{'AF_ESP': '0.00546', 'AF_EXAC': '0.00165', 'A...",Immunodeficiency_38_with_basal_ganglia_calcifi...
1,1,1O14122,542074,C,T,.,.,AF_ESP=0.00015;AF_EXAC=0.00010;ALLELEID=514926...,"{'AF_ESP': '0.00015', 'AF_EXAC': '0.00010', 'A...",Immunodeficiency_38_with_basal_ganglia_calcifi...
2,1,1014143,183381,C,T,.,.,"ALLELEID=181485;CLNDISDB=MedGen:C4015293,OMIM:...","{'ALLELEID': '181485', 'CLNDISDB': 'MedGen:C40...",Immunodeficiency_38_with_basal_ganglia_calcifi...
3,1,1014179,542075,C,T,.,.,"ALLELEID=514896;CLNDISDB=MedGen:C4015293,OMIM:...","{'ALLELEID': '514896', 'CLNDISDB': 'MedGen:C40...",Immunodeficiency_38_with_basal_ganglia_calcifi...
4,1,1014217,475278,C,T,.,.,AF_ESP=0.00515;AF_EXAC=0.00831;AF_TGP=0.00339;...,"{'AF_ESP': '0.00515', 'AF_EXAC': '0.00831', 'A...",Immunodeficiency_38_with_basal_ganglia_calcifi...


In [ ]:
# That worked! So let's pull the other two pieces of additional info I care about. 

clin_v2["GENEINFO"] = clin_v2.INFO_EXT.map(lambda x: x.get('GENEINFO'))
clin_v2["CLNSIG"] = clin_v2.INFO_EXT.map(lambda x: x.get('CLNSIG'))

clin_v2

In [51]:
# Making sure the length didn't change as a result of my transformation. 

assert len(clin_v2) == len(clin_backup)

In [52]:
# Here are the fields we want in the end.
clin_v2[["GENEINFO", "ID", "CHROM", "POS", "REF", "ALT", "CLNSIG", "CLNDN"]].head()


,GENEINFO,ID,CHROM,POS,REF,ALT,CLNSIG,CLNDN
0,ISG15:9636,475283,1,1014O42,G,A,Benign,Immunodeficiency_38_with_basal_ganglia_calcifi...
1,ISG15:9636,542074,1,1O14122,C,T,Uncertain_significance,Immunodeficiency_38_with_basal_ganglia_calcifi...
2,ISG15:9636,183381,1,1014143,C,T,Pathogenic,Immunodeficiency_38_with_basal_ganglia_calcifi...
3,ISG15:9636,542075,1,1014179,C,T,Uncertain_significance,Immunodeficiency_38_with_basal_ganglia_calcifi...
4,ISG15:9636,475278,1,1014217,C,T,Benign,Immunodeficiency_38_with_basal_ganglia_calcifi...


In [53]:
print(len(clin_v2))

102321


In [54]:
# There may be missing values.

clin_v2.apply('count')

CHROM       102321
POS         102321
ID          102321
REF         102321
ALT         102321
FILTER      102321
QUAL        102321
INFO        102321
INFO_EXT    102321
CLNDN        89651
GENEINFO    102307
CLNSIG      100524
dtype: int64

Turns out, there are some values missing in CLNDN, GENEINFO, and CLNSIG. 

Use fillna to fill it with 'Not_Given'. 

And let's overwrite the v2 df with a version that has all the transformations, so it's easier to work with.

In [55]:
clin_v2 = clin_v2[["GENEINFO", "ID", "CHROM", "POS", "REF", "ALT", "CLNSIG", "CLNDN"]].fillna('Not_Given')

In [56]:
# This is what I'll filter now.

clin_v2.head()

,GENEINFO,ID,CHROM,POS,REF,ALT,CLNSIG,CLNDN
0,ISG15:9636,475283,1,1014O42,G,A,Benign,Immunodeficiency_38_with_basal_ganglia_calcifi...
1,ISG15:9636,542074,1,1O14122,C,T,Uncertain_significance,Immunodeficiency_38_with_basal_ganglia_calcifi...
2,ISG15:9636,183381,1,1014143,C,T,Pathogenic,Immunodeficiency_38_with_basal_ganglia_calcifi...
3,ISG15:9636,542075,1,1014179,C,T,Uncertain_significance,Immunodeficiency_38_with_basal_ganglia_calcifi...
4,ISG15:9636,475278,1,1014217,C,T,Benign,Immunodeficiency_38_with_basal_ganglia_calcifi...


In [57]:
sorted(clin_v2.CLNSIG.unique())

['Affects',
 'Affects,_risk_factor',
 'Benign',
 'Benign,_drug_response',
 'Benign,_other',
 'Benign/Likely_benign',
 'Benign/Likely_benign,_association',
 'Benign/Likely_benign,_other',
 'Benign/Likely_benign,_protective',
 'Benign/Likely_benign,_risk_factor',
 'Conflicting_interpretations_of_pathogenicity',
 'Conflicting_interpretations_of_pathogenicity,_Affects',
 'Conflicting_interpretations_of_pathogenicity,_Affects,_association,_drug_response,_other',
 'Conflicting_interpretations_of_pathogenicity,_Affects,_other',
 'Conflicting_interpretations_of_pathogenicity,_association',
 'Conflicting_interpretations_of_pathogenicity,_other',
 'Conflicting_interpretations_of_pathogenicity,_other,_risk_factor',
 'Conflicting_interpretations_of_pathogenicity,_protective',
 'Conflicting_interpretations_of_pathogenicity,_risk_factor',
 'Likely_benign',
 'Likely_benign,_other',
 'Likely_pathogenic',
 'Likely_pathogenic,_association',
 'Likely_pathogenic,_other',
 'Likely_pathogenic,_risk_factor',

I did a quick google search .

* **pathogenic variant**: A genetic alteration that increases an individual’s susceptibility or predisposition to a certain disease or disorder. When such a variant (or mutation) is inherited, development of symptoms is more likely, but not certain. Also called deleterious mutation, disease-causing mutation, predisposing mutation, and susceptibility gene mutation.

Can't find out what drug_response, protective, affects, and risk_factor mean. So I'll focus on the characteristic of "pathogenic".


In [58]:
# Look for clinical significance that contains "pathogenic", but ignore conflicting interpretations. 
# Let's see if my filter works. 

potentially_dangerous = \
    clin_v2[clin_v2.CLNSIG.str.contains("Pathogenic", case=False) 
         & ~clin_v2.CLNSIG.str.contains("Conflicting_interpretations", case=False)
        ]

print("What I deem dangerous:")
print(potentially_dangerous.CLNSIG.unique())


What I deem dangerous:
['Pathogenic' 'Likely_pathogenic' 'Pathogenic/Likely_pathogenic'
 'Pathogenic,_risk_factor' 'Pathogenic/Likely_pathogenic,_risk_factor'
 'Likely_pathogenic,_risk_factor' 'Pathogenic/Likely_pathogenic,_other'
 'Pathogenic,_other' 'Pathogenic,_association,_protective'
 'Likely_pathogenic,_association' 'Pathogenic,_protective'
 'Pathogenic,_Affects' 'Likely_pathogenic,_other']


In [59]:
# Let's see just those rows that I think are NOT THAT dangerous.

clin_v2[~(clin_v2.CLNSIG.str.contains("Pathogenic", case=False) 
        & ~clin_v2.CLNSIG.str.contains("Conflicting_interpretations", case=False))
       ]["CLNSIG"].sort_values().unique()

array(['Affects', 'Affects,_risk_factor', 'Benign',
       'Benign,_drug_response', 'Benign,_other', 'Benign/Likely_benign',
       'Benign/Likely_benign,_association', 'Benign/Likely_benign,_other',
       'Benign/Likely_benign,_protective',
       'Benign/Likely_benign,_risk_factor',
       'Conflicting_interpretations_of_pathogenicity',
       'Conflicting_interpretations_of_pathogenicity,_Affects',
       'Conflicting_interpretations_of_pathogenicity,_Affects,_association,_drug_response,_other',
       'Conflicting_interpretations_of_pathogenicity,_Affects,_other',
       'Conflicting_interpretations_of_pathogenicity,_association',
       'Conflicting_interpretations_of_pathogenicity,_other',
       'Conflicting_interpretations_of_pathogenicity,_other,_risk_factor',
       'Conflicting_interpretations_of_pathogenicity,_protective',
       'Conflicting_interpretations_of_pathogenicity,_risk_factor',
       'Likely_benign', 'Likely_benign,_other', 'Not_Given',
       'Uncertain_signi

In [60]:
# I want to peek at the other columns for non-dangerous mutations. 

clin_v2[~(clin_v2.CLNSIG.str.contains("Pathogenic", case=False) 
        & ~clin_v2.CLNSIG.str.contains("Conflicting_interpretations", case=False))
       ].head(10)

,GENEINFO,ID,CHROM,POS,REF,ALT,CLNSIG,CLNDN
0,ISG15:9636,475283,1,1014O42,G,A,Benign,Immunodeficiency_38_with_basal_ganglia_calcifi...
1,ISG15:9636,542074,1,1O14122,C,T,Uncertain_significance,Immunodeficiency_38_with_basal_ganglia_calcifi...
3,ISG15:9636,542075,1,1014179,C,T,Uncertain_significance,Immunodeficiency_38_with_basal_ganglia_calcifi...
4,ISG15:9636,475278,1,1014217,C,T,Benign,Immunodeficiency_38_with_basal_ganglia_calcifi...
5,ISG15:9636,402986,1,1014228,G,A,Benign,not_specified
6,ISG15:9636,571208,1,1014255,G,A,Uncertain_significance,Immunodeficiency_38_with_basal_ganglia_calcifi...
7,ISG15:9636,568195,1,1014276,G,A,Uncertain_significance,Immunodeficiency_38_with_basal_ganglia_calcifi...
10,ISG15:9636,542076,1,1014401,G,A,Likely_benign,Immunodeficiency_38_with_basal_ganglia_calcifi...
11,ISG15:9636,475281,1,1014451,C,T,Benign,Immunodeficiency_38_with_basal_ganglia_calcifi...
12,ISG15:9636,475282,1,1014471,G,C,Likely_benign,Immunodeficiency_38_with_basal_ganglia_calcifi...


I understand that there are some uncertain significance values or risk factor-related values. For now, I will assume that only the ones coded as Pathogenic or Likely pathogenic as dangerous and relevant for my boss's knowledge.

Also the POS IDs that contained non-numerics - they just happen to not meet my definition of "dangerous" so I don't have to worry about converting this into a numeric value. 

In [61]:
# 1) The deliverables are the final result as a dataframe with a short discussion of any specifics. 
# (that is, what data you would present to your boss with the explanation of your results)

# Dataframes 
result_full = clin_v2[(clin_v2.CLNSIG.str.contains("Pathogenic", case=False) 
                       & ~clin_v2.CLNSIG.str.contains("Conflicting_interpretations", case=False))
                     ]

result_short = clin_v2[(clin_v2.CLNSIG.str.contains("Pathogenic", case=False) 
                        & ~clin_v2.CLNSIG.str.contains("Conflicting_interpretations", case=False))
                       ].head(100)




In [62]:
# Limit your output to the first 100 harmful mutations 
# and tell your boss how many total harmful mutations were found in the file

print("Full result shape:", result_full.shape)

print("Top 100 result shape:", result_short.shape)

Full result shape: (19474, 8)
Top 100 result shape: (100, 8)



# Email to the boss

Hey Boss,

I analyzed the clinvar data and was able to identify some dangerous genetic mutations. 

Please find attached a pandas dataframe called `result_short`. It contains the top 100 rows from the full dataset, which is in the dataframe called `result_full`. 

There were a total of 19,474 mutations that I think are dangerous. Having zero backbround in biology, I defined "dangerous" as any mutation marked as either pathogenic or likely pathogenic in the dataset's additional information (`INFO`), under its clinical significance field (`CLNSIG`). I excluded any mutations with clinical significance that was either uncertain or having conflicting interpretations. Let me know if you want me to include them.

As you requested, `result_short` should have the following fields:

|Field name|Description|Comments|
|---|---|---|
|GENEINFO|Gene name  ||
|ID|Mutation ID number  ||
|CHROM|Mutation Position (chromosome & position)  ||
|POS|Mutation Position (chromosome & position)  ||
|REF|Mutation value (reference & alternate bases)  |Reference base|
|ALT|Mutation value (reference & alternate bases)  |Alternate base|
|CLNSIG|Clinical severity  |Filtered for pathogenic, excluding uncertain or conflicting interpretations|
|CLNDN|Disease that is implicated|Not code, but English names you can read.|

See below for the table output of the top 100 rows. Please let me know if you have any questions. Thanks!

Best,
Haerang


In [63]:
result_short

,GENEINFO,ID,CHROM,POS,REF,ALT,CLNSIG,CLNDN
2,ISG15:9636,183381,1,1014143,C,T,Pathogenic,Immunodeficiency_38_with_basal_ganglia_calcifi...
8,ISG15:9636,161455,1,1014316,C,CG,Pathogenic,Immunodeficiency_38_with_basal_ganglia_calcifi...
9,ISG15:9636,161454,1,1014359,G,T,Pathogenic,Immunodeficiency_38_with_basal_ganglia_calcifi...
24,AGRN:375790,243036,1,1022225,G,A,Pathogenic,Congenital_myasthenic_syndrome
26,AGRN:375790,243037,1,1022313,A,T,Pathogenic,Congenital_myasthenic_syndrome
46,AGRN:375790,574478,1,1041354,CGCCCGCCAGGAGAATGTCTTCAAGAAGTTCGACG,C,Pathogenic,"Myasthenic_syndrome,_congenital,_8"
49,AGRN:375790,126556,1,1041582,C,T,Pathogenic,Congenital_myasthenic_syndrome|Myasthenic_synd...
63,AGRN:375790,243038,1,1042136,T,TC,Pathogenic,Congenital_myasthenic_syndrome
237,AGRN:375790,489335,1,1049672,C,T,Likely_pathogenic,Not_Given
270,AGRN:375790,243039,1,1050473,G,A,Pathogenic,Congenital_myasthenic_syndrome
